# __<center>ECE421 - Winter 2020__
# __<center> Assignment 1:__
# **<center>Linear and Logistic Regression**

Haotian Luo 1004102865 <br>
Yuheng Wang

<br>



## 1. Linear Regression
### 1.1 Loss Function and Gradient

In [0]:
def loadData():
    with np.load('notMNIST.npz') as data :
        Data, Target = data ['images'], data['labels']
        posClass = 2
        negClass = 9
        dataIndx = (Target==posClass) + (Target==negClass)
        Data = Data[dataIndx]/255.
        Target = Target[dataIndx].reshape(-1, 1)
        Target[Target==posClass] = 1
        Target[Target==negClass] = 0
        np.random.seed(421)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data, Target = Data[randIndx], Target[randIndx]
        trainData, trainTarget = Data[:3500], Target[:3500]
        validData, validTarget = Data[3500:3600], Target[3500:3600]
        testData, testTarget = Data[3600:], Target[3600:]
    return trainData, validData, testData, trainTarget, validTarget, testTarget

def MSE(W, b, x, y, reg):
    y_hat = np.matmul(x, W) + b
    mse_loss = (np.linalg.norm(y_hat - y)) ** 2
    wd_loss = (np.linalg.norm(W) ** 2) * reg / 2

    return (mse_loss + wd_loss)
    
def gradMSE(W, b, x, y, reg):
    N = len(y)
    y_hat = np.matmul(x, W) + b
    grad_weight = (1/N) * np.dot(np.transpose(x), np.subtract(y_hat, y)) + reg * W
    grad_bias = (1/N) * np.sum(y_hat - y)

    return grad_weight, grad_bias

### 1.2 Gradient Descent Implementation

In [0]:
#def timer(method):
#    def timed(*args, **kwargs):
#        ts = time.time()
#        result = method(*args, **kwargs)
#        te = time.time()
#        print (method.__name__, ':', (te - ts) * 1000, 'ms')
#        return result
#    return timed

#@timer
def grad_descent(W, b, x, y, alpha, epochs, reg, error_tol, lossType = "MSE"):
    weight_record = []
    bias_record = []
    loss_record = []

    print("Calculating gradient descent of", lossType, "with learning rate =", alpha, "and regularizer =", reg)
    current_weight = W.reshape(x.shape[1], 1)
    current_bias = b
    
    if lossType is "MSE":
        for _ in range(0, epochs):
            grad_w, grad_b = gradMSE(current_weight, current_bias, x, y, reg)
            updated_weight = current_weight - np.multiply(alpha, grad_w)
            if (np.linalg.norm(updated_weight - current_weight) < error_tol):
                break
            updated_bias = current_bias - np.multiply(alpha, grad_b)
            loss = MSE(updated_weight, updated_bias, x, y, reg)
            current_weight = updated_weight
            current_bias = updated_bias
            weight_record.append(updated_weight)
            bias_record.append(updated_bias)
            loss_record.append(loss)
    elif lossType is "CE":
        for _ in range(0, epochs):
            grad_w, grad_b = gradCE(current_weight, current_bias, x, y, reg)
            updated_weight = current_weight - np.multiply(alpha, grad_w)
            if (np.linalg.norm(updated_weight - current_weight) < error_tol):
                break
            updated_bias = current_bias - np.multiply(alpha, grad_b)
            loss = crossEntropyLoss(updated_weight, updated_bias, x, y, reg)
            current_weight = updated_weight
            current_bias = updated_bias
            weight_record.append(updated_weight)
            bias_record.append(updated_bias)
            loss_record.append(loss)

    else:
        exit -1
    return weight_record, bias_record, loss_record

    
def crossEntropyLoss(W, b, x, y, reg):
    N,n = x.shape
    y_hat = 1./(1 + np.exp(-(np.dot(x,W) + b)))
    total_loss = 1/N * (np.sum(-np.multiply(y, np.log(y_hat) - np.multiply((1 - y), np.log(y_hat))))) + reg/2 * (np.linalg.norm(W) ** 2)
    return total_loss

def gradCE(W, b, x, y, reg):
    N,n = x.shape
    y_hat = 1./(1 + np.exp(-(np.dot(x,W) + b)))
    grad_weight = 1/N * np.dot(x.T, y - y_hat) + reg * np.linalg.norm(W)
    grad_bias = 1/N * np.sum(y - y_hat)
    return grad_weight, grad_bias

def accuracy_calculation(W, b, x, y):
    acc = [np.sum((np.dot(x, W[i]) + b[i] >= 0.5) == y) / y.shape[0] for i in range(len(W))]
    return acc
    
def buildGraph(loss="MSE"):
    #Initialize weight and bias tensors
    tf.set_random_seed(421)

    if loss == "MSE":
    # Your implementation
        return

    elif loss == "CE":
    #Your implementation here
        return

### 1.3 Tuning the Learning Rate

In the below chart, data is trained with Batch Gradient Descent Algorithm. We fix the regularizer $\lambda$ = 0, and we change the learning rate $\alpha$ = {0.005, 0.001, 0.0001}. The final *Mean Square Error, Trainning Accuracy, Validation Accuracy, Testing Accuracy, and Computation Time* are measured and recorded.

| $\alpha$ = |MSE|Training Accuracy|Validation Accuracy|Testing Accuracy|Computation Time|
|:-:|:-:|:-:|:-:|:-:|:-:|
|0.005|7072.130|0.644|0.61|0.634|16390 ms|
|0.001|23569.585|0.572|0.56|0.476|15685 ms|
|0.0001|77365.574|0.527|0.54|0.524|15843 ms|

### 1.4 Generalization

In the below chart, data is trained with Batch Gradient Descent Algorithm. We fix the learning rate $\alpha$ = 0.005, and we change the regularizer $\lambda$ = {0.001, 0.1, 0.5}. The final *Mean Square Error, Trainning Accuracy, Validation Accuracy, Testing Accuracy, and Computation Time* are measured and recorded.

| $\lambda$ = |MSE|Training Accuracy|Validation Accuracy|Testing Accuracy|Computation Time|
|:-:|:-:|:-:|:-:|:-:|:-:|
|0.001|6733.505|0.648|0.61|0.634|15726 ms|
|0.1|158.814|0.962|0.97|0.966|15416 ms|
|0.5|121.288|0.968|0.97|0.966|15581 ms|

### 1.5 Comparing Batch GD with normal equation

In the below chart, we calculate the optimum weight using "normal equation" of the **least sqaures** formula. 
$$ \underline{W}_{LS} = (X^TX + \lambda I)^{-1} X^T Y $$

In [0]:
def normal_equation (x, y, reg):
    data_size = x.shape[0]
    dimension = x.shape[1]
    x = np.append(np.ones((data_size, 1)), x, axis = 1)

    I = np.identity(dimension + 1)
    I[:, 0] = 0

    w_normal = np.dot(np.dot(np.linalg.inv(np.dot(np.transpose(x), x) + reg * I), np.transpose(x)), y)
    b_normal = w_normal[0]
    w_normal = np.delete(w_normal, 0).reshape(dimension, 1)
    
    return w_normal, b_normal

We change the regularizer  λ  = {0, 0.001, 0.1, 0.5}. The final *Mean Square Error, Trainning Accuracy, Validation Accuracy, Testing Accuracy, and Computation Time* are measured and recorded.

| $\lambda$ = |MSE|Training Accuracy|Validation Accuracy|Testing Accuracy|Computation Time|
|:-:|:-:|:-:|:-:|:-:|:-:|
|0|65.464|0.994|0.96|0.945|340 ms|
|0.001|65.470|0.994|0.96|0.945|299 ms|
|0.1|158.814|0.994|0.97|0.945|299 ms|
|0.5|121.288|0.993|0.97|0.945|300 ms|

Then, we compare the loss, accuracy, and computation time between **Batch GD Algorithm** and **Normal Equation**. The learning rate $\alpha$ of Batch GD is chosen to be 0.005 in each case.

| $\lambda$ = |Method: B/N|MSE|Training Accuracy|Validation Accuracy|Testing Accuracy|Computation Time|
|:-:|:-:|:-:|:-:|:-:|:-:|:-:|
|0|Batch GD|7072.130|0.644|0.61|0.634|16390 ms|
||Normal Eq|65.464|0.994|0.96|0.945|340 ms|
|0.001|Batch GD|6733.505|0.648|0.61|0.634|15726 ms|
||Normal Eq|65.470|0.994|0.96|0.945|299 ms|
|0.1|Batch GD|158.814|0.962|0.97|0.966|15416 ms|
||Normal Eq|158.814|0.994|0.97|0.945|299 ms|
|0.5|Batch GD|121.288|0.968|0.97|0.966|15581 ms|
||Normal Eq|121.288|0.993|0.97|0.945|300 ms|

In [0]:
%tensorflow_version 2.x

import matplotlib.pyplot as plt
import numpy as np
import time

def print_info(loss, train_a, valid_a, test_a, type, alpha, reg, comp_time):    
    if type is "GD":
        print('Batch GD with \u03B1 = {}, \u03BB = {}, MSE = {}, training accuracy = {}, valid accuracy = {}, test accuracy = {}, '
            'computation time = {} ms'.format(alpha, reg, loss, train_a, valid_a, test_a, int(comp_time * 1000)))
    elif type is "normal":
        print('Normal Equation with \u03BB = {}, MSE = {}, training accuracy = {}, valid accuracy = {}, test accuracy = {}, '
            'computation time = {} ms'.format(reg, loss, train_a, valid_a, test_a, int(comp_time * 1000)))

def linreg():
    #1.3

    import time

    trainData, validData, testData, trainTarget, validTarget, testTarget = loadData()
    trainData = np.array([x.flatten() for x in trainData])
    validData = np.array([x.flatten() for x in validData])
    testData = np.array([x.flatten() for x in testData])


    alpha1, alpha2, alpha3 = 0.005, 0.001, 0.0001
    reg = 0
    epochs = 5000
    error_tol = 1e-7

    init_weight = np.random.normal(size=(784,1))
    init_bias = np.random.uniform(-1,1)

    start1 = time.time()
    weight_train1, bias_train1, loss_train1 = grad_descent(init_weight, init_bias, trainData, trainTarget, alpha1, epochs, reg, error_tol)
    end1 = time.time()
    start2 = time.time()
    weight_train2, bias_train2, loss_train2 = grad_descent(init_weight, init_bias, trainData, trainTarget, alpha2, epochs, reg, error_tol)
    end2 = time.time()
    start3 = time.time()
    weight_train3, bias_train3, loss_train3 = grad_descent(init_weight, init_bias, trainData, trainTarget, alpha3, epochs, reg, error_tol)
    end3 = time.time()


    accuracy_train1 = accuracy_calculation(weight_train1, bias_train1, trainData, trainTarget)
    accuracy_train2 = accuracy_calculation(weight_train2, bias_train2, trainData, trainTarget)
    accuracy_train3 = accuracy_calculation(weight_train3, bias_train3, trainData, trainTarget)

    plt.figure()
    plt.suptitle('Training losses')
    plt.plot(loss_train1,'',loss_train2,'',loss_train3,'')
    plt.xlabel('epochs')
    plt.ylabel('losses')
    plt.grid()
    plt.legend(['MSE: \u03B1 = {}, \u03BB = {}'.format(alpha1, reg),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha2, reg),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha3, reg)])
    plt.savefig('Learning_rate_adjustment_training_loss_LinReg.png')


    plt.figure()
    plt.suptitle('Training accuracy')
    plt.plot(accuracy_train1,'',accuracy_train2,'',accuracy_train3,'')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['MSE: \u03B1 = {}, \u03BB = {}'.format(alpha1, reg),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha2, reg),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha3, reg)])
    plt.savefig('Learning_rate_adjustment_training_accuracy_LinReg.png')

    accuracy_valid1 = accuracy_calculation(weight_train1, bias_train1, validData, validTarget)
    accuracy_valid2 = accuracy_calculation(weight_train2, bias_train2, validData, validTarget)
    accuracy_valid3 = accuracy_calculation(weight_train3, bias_train3, validData, validTarget)

    plt.figure()
    plt.suptitle('Validation accuracy')
    plt.plot(accuracy_valid1,'',accuracy_valid2,'',accuracy_valid3,'')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['MSE: \u03B1 = {}, \u03BB = {}'.format(alpha1, reg),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha2, reg),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha3, reg)])
    plt.savefig('Learning_rate_adjustment_validation_accuracy_LinReg.png')

    accuracy_test1 = accuracy_calculation(weight_train1, bias_train1, testData, testTarget)
    accuracy_test2 = accuracy_calculation(weight_train2, bias_train2, testData, testTarget)
    accuracy_test3 = accuracy_calculation(weight_train3, bias_train3, testData, testTarget)

    plt.figure()
    plt.suptitle('Testing accuracy')
    plt.plot(accuracy_test1,'',accuracy_test2,'',accuracy_test3,'')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['MSE: \u03B1 = {}, \u03BB = {}'.format(alpha1, reg),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha2, reg),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha3, reg)])
    plt.savefig('Learning_rate_adjustment_testing_accuracy_LinReg.png')

    print_info(loss_train1[-1], accuracy_train1[-1], accuracy_valid1[-1], accuracy_test1[-1], "GD", alpha1, reg, end1 - start1)
    print_info(loss_train2[-1], accuracy_train2[-1], accuracy_valid2[-1], accuracy_test2[-1], "GD", alpha2, reg, end2 - start2)
    print_info(loss_train3[-1], accuracy_train3[-1], accuracy_valid3[-1], accuracy_test3[-1], "GD", alpha3, reg, end3 - start3)


    #1.4
    alpha = 0.005
    reg1, reg2, reg3 = 0.001, 0.1, 0.5
    
    start4 = time.time()
    weight_train4, bias_train4, loss_train4 = grad_descent(init_weight, init_bias, trainData, trainTarget, alpha, epochs, reg1, error_tol)
    end4 = time.time()
    start5 = time.time()
    weight_train5, bias_train5, loss_train5 = grad_descent(init_weight, init_bias, trainData, trainTarget, alpha, epochs, reg2, error_tol)
    end5 = time.time()
    start6 = time.time()
    weight_train6, bias_train6, loss_train6 = grad_descent(init_weight, init_bias, trainData, trainTarget, alpha, epochs, reg3, error_tol)
    end6 = time.time()


    accuracy_train4 = accuracy_calculation(weight_train4, bias_train4, trainData, trainTarget)
    accuracy_train5 = accuracy_calculation(weight_train5, bias_train5, trainData, trainTarget)
    accuracy_train6 = accuracy_calculation(weight_train6, bias_train6, trainData, trainTarget)

    plt.figure()
    plt.suptitle('Training losses')
    plt.plot(loss_train4,'',loss_train5,'',loss_train6,'')
    plt.xlabel('epochs')
    plt.ylabel('losses')
    plt.grid()
    plt.legend(['MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg1),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg2),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg3)])
    plt.savefig('Regulation_adjustment_training_loss_LinReg.png')


    plt.figure()
    plt.suptitle('Training accuracy')
    plt.plot(accuracy_train4,'',accuracy_train5,'',accuracy_train6,'')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg1),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg2),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg3)])
    plt.savefig('Regulation_adjustment_training_accuracy_LinReg.png')

    accuracy_valid4 = accuracy_calculation(weight_train4, bias_train4, validData, validTarget)
    accuracy_valid5 = accuracy_calculation(weight_train5, bias_train5, validData, validTarget)
    accuracy_valid6 = accuracy_calculation(weight_train6, bias_train6, validData, validTarget)

    plt.figure()
    plt.suptitle('Validation accuracy')
    plt.plot(accuracy_valid4,'',accuracy_valid5,'',accuracy_valid6,'')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg1),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg2),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg3)])
    plt.savefig('Regulation_adjustment_validation_accuracy_LinReg.png')

    accuracy_test4 = accuracy_calculation(weight_train4, bias_train4, testData, testTarget)
    accuracy_test5 = accuracy_calculation(weight_train5, bias_train5, testData, testTarget)
    accuracy_test6 = accuracy_calculation(weight_train6, bias_train6, testData, testTarget)

    plt.figure()
    plt.suptitle('Testing accuracy')
    plt.plot(accuracy_test4,'',accuracy_test5,'',accuracy_test6,'')
    plt.xlabel('epochs')
    plt.ylabel('accuracy')
    plt.grid()
    plt.legend(['MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg1),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg2),'MSE: \u03B1 = {}, \u03BB = {}'.format(alpha, reg3)])
    plt.savefig('Regulation_adjustment_testing_accuracy_LinReg.png')

    print_info(loss_train4[-1], accuracy_train4[-1], accuracy_valid4[-1], accuracy_test4[-1], "GD", alpha, reg1, end4 - start4)
    print_info(loss_train5[-1], accuracy_train5[-1], accuracy_valid5[-1], accuracy_test5[-1], "GD", alpha, reg2, end5 - start5)
    print_info(loss_train6[-1], accuracy_train6[-1], accuracy_valid6[-1], accuracy_test6[-1], "GD", alpha, reg3, end6 - start6)


    #1.5
    start = time.time()
    w_normal_train, b_normal_train = normal_equation(trainData, trainTarget, reg)
    end = time.time()
    start1 = time.time()
    w_normal_train1, b_normal_train1 = normal_equation(trainData, trainTarget, reg1)
    end1 = time.time()
    start2 = time.time()
    w_normal_train2, b_normal_train2 = normal_equation(trainData, trainTarget, reg2)
    end2 = time.time()
    start3 = time.time()
    w_normal_train3, b_normal_train3 = normal_equation(trainData, trainTarget, reg3)
    end3 = time.time()
    
    loss_normal = MSE(w_normal_train, b_normal_train, trainData, trainTarget, reg)
    loss_normal1 = MSE(w_normal_train1, b_normal_train1, trainData, trainTarget, reg1)
    loss_normal2 = MSE(w_normal_train2, b_normal_train2, trainData, trainTarget, reg2)
    loss_normal3 = MSE(w_normal_train3, b_normal_train3, trainData, trainTarget, reg3)
    
    w_normal_train = [w_normal_train]
    w_normal_train1 = [w_normal_train1]
    w_normal_train2 = [w_normal_train2]
    w_normal_train3 = [w_normal_train3]
    b_normal_train = [b_normal_train]
    b_normal_train1 = [b_normal_train1]
    b_normal_train2 = [b_normal_train2]
    b_normal_train3 = [b_normal_train3]

    normal_accuracy_train = accuracy_calculation(w_normal_train, b_normal_train, trainData, trainTarget)
    normal_accuracy_train1 = accuracy_calculation(w_normal_train1, b_normal_train1, trainData, trainTarget)
    normal_accuracy_train2 = accuracy_calculation(w_normal_train2, b_normal_train2, trainData, trainTarget)
    normal_accuracy_train3 = accuracy_calculation(w_normal_train3, b_normal_train3, trainData, trainTarget)

    normal_accuracy_valid = accuracy_calculation(w_normal_train, b_normal_train, validData, validTarget)
    normal_accuracy_valid1 = accuracy_calculation(w_normal_train1, b_normal_train1, validData, validTarget)
    normal_accuracy_valid2 = accuracy_calculation(w_normal_train2, b_normal_train2, validData, validTarget)
    normal_accuracy_valid3 = accuracy_calculation(w_normal_train3, b_normal_train3, validData, validTarget)

    normal_accuracy_test = accuracy_calculation(w_normal_train, b_normal_train, testData, testTarget)
    normal_accuracy_test1 = accuracy_calculation(w_normal_train1, b_normal_train1, testData, testTarget)
    normal_accuracy_test2 = accuracy_calculation(w_normal_train2, b_normal_train2, testData, testTarget)
    normal_accuracy_test3 = accuracy_calculation(w_normal_train3, b_normal_train3, testData, testTarget)

    print_info(loss_normal, normal_accuracy_train[0], normal_accuracy_valid[0], normal_accuracy_test[0], "normal", 0, reg, end - start)
    print_info(loss_normal1, normal_accuracy_train1[0], normal_accuracy_valid1[0], normal_accuracy_test1[0], "normal", 0, reg1, end1 - start1)
    print_info(loss_normal2, normal_accuracy_train2[0], normal_accuracy_valid2[0], normal_accuracy_test2[0], "normal", 0, reg2, end2 - start2)
    print_info(loss_normal3, normal_accuracy_train3[0], normal_accuracy_valid3[0], normal_accuracy_test3[0], "normal", 0, reg3, end3 - start3)
    

linreg()